# Kronecker Theta Functions and Kronecker Double Series

This notebook aims to setup code for exploring Kronecker Theta Functions and Kronecker Double Series. The aims are to:

- give defintitions of the Kronecker Theta function in terms of ratios of Jacobi Theta functions and Weierstrass Sigma functions.
- numerically verify that the functions can be correctly plotted and tested.

The motivation for setting this up will become apparent in other notebbooks which aim to:
- explore the series expansions in some dynamic systems with known sigma function ratio solutions and see if they yield new insights.
- ultimately demonstrate the importance of the Kronecker theta function in nonlinear coupled ordinary differential equations and associated applications.

**Important References**

The term *Kronecker theta function* for the ratio of Jacobi theta functions was first heard by me in 2023 when reading the 2016 publication 
[*Elliptic Functions According to Eisenstein and Kronecker: An Update*](https://webusers.imj-prg.fr/~pierre.charollois/Charollois-Sczech_EMS.pdf) by Pierre Charollois and Robert Sczech. They attribute the term to [*Algebraic theta functions and p-adic interpolation of Eisenstein-Kronecker numbers*](https://arxiv.org/abs/math/0610163) by Kenichi Bannai and Shinichi Kobayashi published in 2006. It was the 1976 Andre Weil book that rekindled modern interest in this area of Kronecker's work and inspired the Charollois and Sczech update, and teh Weil book is based on some notes of Kroneckers lectures written up by F. von Dalwigk in 1891 that expanded on work by Eisenstein. Those notes were digitised and made available in German [here](https://webusers.imj-prg.fr/~pierre.charollois/Kronecker.html). It is also worth mentioning that in [*KRONECKER THETA FUNCTION AND A DECOMPOSITION THEOREM FOR THETA FUNCTIONS I*](https://arxiv.org/pdf/2012.01670.pdf) published in 2020 by ZHI-GUO LIU it was shown that meromorphic functions that satisfy certain conditions re poles and functional identities also satisfy a decomposition theorem in terms of Kronecker theta functions. Finally, it is to be noted that part of the work in Charollois and Sczech follows the 2006 paper [*Algebraic theta functions and p-adic interpolation of Eisenstein-Kronecker numbers*](https://arxiv.org/pdf/math/0610163.pdf) by Kenichi Bannai, Shinichi Kobayashi.

I was aware that the theta or sigma ratio could be expanded in a *Kronecker double series* and mentioned this in an appendix in the 2015 paper [*General complex envelope solutions of coupled-mode optics with quadratic or cubic nonlinearity*](https://arxiv.org/abs/1512.03092) and in that paper cited two sources for that claim one was *Elliptic Functions according to Eisenstein and Kronecker*, by Andre Weil, published in Springer-Verlag, 1972, and the other was *E. T. Whitaker and G. N. Watson, A Course of Modern Analysis, (Merchant Books, 1915)* (5th edition, excercise 20.34 p20.34 where it is attributed to Math. Trip. 1895). I mentioned the double series in the Appendix as I thought a Fourier series could prove useful in optics but never got round to exploring it. That said, the form of the double series mentioned in my Appendix is not the same as the one in Charollois and Sczech, and it is theirs that has once again peaked my interest as it is a pole expansion that could prove useful in physical interpretations and perhaps even be used as an ansatz to find new solutions to dynamic systems that have so far proved illusive. This work will begin my exploration of the Kronecker theta function as a double series pole expansion.

## Setting up the variables

In [90]:
from sympy import *
(
   xi, eta, u, tau, N, M, n, m, k, l, z, x, y, t, q, a, w, s, g2, g3, omega1, omega2, omega3, kappa, rho
) = symbols(
'''xi, eta, u, tau, N, M, n, m, k, l, z, x, y, t, q, a, w, s, g2, g3, omega1, omega2, omega3, kappa, rho'''
)
x1, x2, x3, x4, x5, x6 = symbols('x1, x2, x3, x4, x5, x6', real=True)
Ser = Function('Ser')
pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass Zeta function
sigma = Function('sigma') # Weierstrass Sigma function
eta = Function('eta') # Dedekind eta function (used in Kronecker second limit formula)
f = Function('f')
h = Function('h')
K = Function('K')
theta = Function('theta')
Theta = Function('Theta')

vartheta = IndexedBase('vartheta')
from math import prod

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from numpy import linspace, absolute, angle, square, real, imag, conj, array as arraynp, vectorize, concatenate
import scipy.integrate
import matplotlib.pyplot as plt

# The package containing mpmath expressions for Weierstrass elliptic functions
from numerical_evaluation.weierstrass_modified import Weierstrass
we = Weierstrass()
from mpmath import exp as mpexp
from mpmath import jtheta, mpf
def mpc_to_float(mpc_val):
    return float(mpc_val.real) + float(mpc_val.imag)*1j

# %load_ext autoreload
# %autoreload 2

## The Kronecker theta function as a double series

### Kronecker double series definition
Starting from equation $(5)$ in Charollois and Sczech (in the limit that $N,M$ go to $\infty$). I am using $\rho$ where they used the variable $\eta$ because they later define the function $\eta(\tau)$ in the same function which confuses matters. In this notebook $\eta(\tau)$ denotes the Dedekind eta function.

In [60]:
Seq_dbl_series = Eq(Ser(xi, rho, u, tau), Sum(Sum(exp(2*I*pi*(-m*xi + n*rho))/(u + m + n*tau),(n,-N,N)),(m,-M,M)))
Seq_dbl_series

Eq(Ser(xi, rho, u, tau), Sum(exp(2*I*pi*(-m*xi + n*rho))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

### Jacobi theta function definition

Here the Jacobi theta function is defned and it will be denoted using $\theta$ as sympy is somewhat restricted by characters for functions (although vartheta works as a symbol). This theta function is intended to be the same Jacobi theta function used in Charollois and Sczech and it is commonly denoted $\mathcal{theta}_1$ elsewhere, e.g. https://mathworld.wolfram.com/JacobiThetaFunctions.html. It will be defined using the same series and products used in Charollois and Sczech. Furthermore, checks are performed to also confirm that the value coming from the $\mathrm{jtheta}(1,\pi z,\exp(i \pi \tau))$ function in the mpmath python package also agrees with the definition, which is important as that forms part of the numerical evaluation of Weierstrass elliptic functions used elsewhere.

In [4]:
Eq(theta(z,tau), vartheta[1])
tau_domain = im(tau)>0
q_tau = Eq(q, exp(2*I*pi*tau))
theta_sum_def = Eq(theta(z,tau), Sum(exp(2*I*pi*((n + Rational(1,2))**2*tau/2 + (n + Rational(1,2))*(z - Rational(1,2)))),(n,-N,N)))
theta_prod_def = Eq(theta(z,tau), 2*exp(I*pi*tau/4)*sin(pi*z)*Product((1-q**n)*(1-q**n*exp(2*I*pi*z))*(1-q**n*exp(-2*I*pi*z)),(n,1,N)))
tau_domain
q_tau
theta_sum_def
theta_prod_def

Eq(theta(z, tau), vartheta[1])

im(tau) > 0

Eq(q, exp(2*I*pi*tau))

Eq(theta(z, tau), Sum(exp(2*I*pi*(tau*(n + 1/2)**2/2 + (n + 1/2)*(z - 1/2))), (n, -N, N)))

Eq(theta(z, tau), 2*exp(I*pi*tau/4)*sin(pi*z)*Product((1 - q**n)*(-q**n*exp(-2*I*pi*z) + 1)*(-q**n*exp(2*I*pi*z) + 1), (n, 1, N)))

In [61]:
N_val = 200
z_val = 2.245
tau_val = 0.78+I*0.8
test_params = [(N,N_val),(z,z_val), (tau, tau_val)]
Eq(theta_sum_def.lhs.subs(test_params), theta_sum_def.rhs.subs(test_params).doit().evalf())
Eq(theta_prod_def.lhs.subs(test_params), theta_prod_def.rhs.subs(*q_tau.args).subs(test_params).doit().evalf())
Eq(theta(z_val, tau_val), mpc_to_float(jtheta(1,(z_val*pi).evalf(), exp(I*pi*tau_val).evalf())))

Eq(theta(2.245, 0.78 + 0.8*I), 0.603776148725747 + 0.43055767622146*I)

Eq(theta(2.245, 0.78 + 0.8*I), 0.603776148725689 + 0.430557676221541*I)

Eq(theta(2.245, 0.78 + 0.8*I), 0.603776148725747 + 0.43055767622146*I)

#### Derivatives of the Jacobi theta functions
The notation from mpmath python package to denote derivatives using a third paramter will be adopted here and the mpmath package will be used to evaluate them, that is:

$\theta(z,\tau,k)=\pi^k \mathrm{jtheta}(1,\pi z,\exp(i \pi \tau),k)$

and let:

In [6]:
theta_z_k_dif = Eq(theta(z,tau,k), Derivative(theta(z,tau),(z,k)))
theta_z_k_dif

Eq(theta(z, tau, k), Derivative(theta(z, tau), (z, k)))

### Kronecker's theorem relating the Kronecker theta function to the Kronecker double series
Kronecker's main result (which is Theorem 2.4 in Charollois and Sczech) is to express the ratio of Jacobi theta functions in terms of the double series as follows:

In [62]:
0<im(u)
im(u)<im(tau)
0<xi
xi<1
Eq(im(xi),0)
Eq(im(rho),0)

im(u) > 0

im(u) < im(tau)

xi > 0

xi < 1

Eq(im(xi), 0)

Eq(im(rho), 0)

In [63]:
Seq_dbl_ktheta = Eq(Ser(xi, rho, u, tau), exp(2*I*pi*xi*u)*theta(0,tau,1)*theta(u + rho + tau*xi, tau)/theta(u, tau)/theta(rho + tau*xi, tau))
Seq_dbl_ktheta

Eq(Ser(xi, rho, u, tau), theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(rho + tau*xi, tau)))

In [64]:
ktheta_dbl_series = Seq_dbl_series.subs(*Seq_dbl_ktheta.args)
ktheta_dbl_series

Eq(theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(rho + tau*xi, tau)), Sum(exp(2*I*pi*(-m*xi + n*rho))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [208]:
ktheta_dbl_series.args

(theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(rho + tau*xi, tau)),
 Sum(exp(2*I*pi*(-m*xi + n*rho))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [69]:
rho_val = 3.78
tau_val = 8.239765 + 2.3*I
u_val = 4.34144 + 0.8*I
xi_val = 0.435
N_val = 100
M_val = 100
S_dbl_subs_rho_tau_u_xi = [(rho, rho_val), (tau, tau_val), (u, u_val), (xi, xi_val), (N, N_val), (M, M_val)]

In [70]:
def theta_ratio_S(_xi, _rho, _u, _tau):
    result = (
        exp(2*I*pi*_u*_xi) * 
        jtheta(1,((_rho + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf()) *
        pi * jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) * jtheta(1,((_rho + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) )
    ).evalf()
    return result

def theta_ratio_S_u_diff(_xi, _rho, _u, _tau):
    result = (
        2*I*pi**2*_xi*exp(2*I*pi*_u*_xi) * 
        jtheta(1,((_rho + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf()) *
        jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) * jtheta(1,((_rho + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) ) +

        pi**2*exp(2*I*pi*_u*_xi) * jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) * 
        jtheta(1,((_rho + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) * jtheta(1,((_rho + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) ) -

        pi**2*exp(2*I*pi*_u*_xi) * jtheta(1,((_rho + _tau * _xi + _u) * pi).evalf(), exp(I*pi*_tau).evalf()) *
        jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) * jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,(_u * pi).evalf(), exp(I*pi*_tau).evalf()) ** 2 * jtheta(1,((_rho + _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf()) ) 

    ).evalf()
    return result

def dbl_series_S_summand(_xi, _rho, _u, _tau, _m, _n, _s):
    return (
        exp(2*I*pi*(- _m * _xi + _n * _rho))/(_u + _m + _n * _tau) ** _s
    ).evalf()

def dbl_series_S(_xi, _rho, _u, _tau, _M, _N, _s=1):
    result = 0
    for _m in range(-_M, _M + 1):
        for _n in range(-_N, _N + 1):
            result += (-1) ** (_s+1) * dbl_series_S_summand(_xi, _rho, _u, _tau, _m, _n, _s)
    return result

In [71]:
S_val_1 = Eq(Seq_dbl_ktheta.lhs, theta_ratio_S(xi_val, rho_val, u_val, tau_val))
S_val_2 = Eq(Seq_dbl_series.lhs, dbl_series_S(xi_val, rho_val, u_val, tau_val, M_val, N_val))
S_val_1
S_val_2
abs(S_val_1.rhs - S_val_2.rhs) / abs(S_val_1.rhs + S_val_2.rhs) / 2

Eq(Ser(xi, rho, u, tau), -0.463220020983809 - 0.567076331077644*I)

Eq(Ser(xi, rho, u, tau), -0.462091113701485 - 0.573383065191033*I)

0.00218128905779438

#### Checking the derivative also as it will converge faster

In [21]:
Eq(Derivative(ktheta_dbl_series.lhs,u), Derivative(ktheta_dbl_series.rhs,u))
Eq(diff(ktheta_dbl_series.lhs,u), diff(ktheta_dbl_series.rhs,u))

Eq(Derivative(theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)), u), Derivative(Sum(exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u), (n, -N, N), (m, -M, M)), u))

Eq(2*I*pi*xi*theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(eta + tau*xi, tau)) + theta(0, tau, 1)*exp(2*I*pi*u*xi)*Subs(Derivative(theta(_xi_1, tau), _xi_1), _xi_1, eta + tau*xi + u)/(theta(u, tau)*theta(eta + tau*xi, tau)) - theta(eta + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)*Derivative(theta(u, tau), u)/(theta(u, tau)**2*theta(eta + tau*xi, tau)), Sum(-exp(2*I*pi*(eta*n - m*xi))/(m + n*tau + u)**2, (n, -N, N), (m, -M, M)))

In [72]:
Eq((theta(0, tau, 1)*exp(2*I*pi*u*xi)*theta(rho + tau*xi + u, tau, 1)/(theta(u, tau)*theta(rho + tau*xi, tau))
 -theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)*theta(u, tau,1)/(theta(u, tau)**2*theta(rho + tau*xi, tau))+
 2*I*pi*xi*theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(rho + tau*xi, tau))),
   -Sum(Sum(exp(2*I*pi*(-m*xi + n*rho))/(u + m + n*tau)**2,(n,-N,N)),(m,-M,M)))

Eq(2*I*pi*xi*theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(rho + tau*xi, tau)) + theta(0, tau, 1)*theta(rho + tau*xi + u, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)*theta(rho + tau*xi, tau)) - theta(rho + tau*xi + u, tau)*theta(0, tau, 1)*theta(u, tau, 1)*exp(2*I*pi*u*xi)/(theta(u, tau)**2*theta(rho + tau*xi, tau)), -Sum(exp(2*I*pi*(-m*xi + n*rho))/(m + n*tau + u)**2, (n, -N, N), (m, -M, M)))

In [73]:
S_val_1_diff_u = Eq(Derivative(Seq_dbl_ktheta.lhs,u), 
                    theta_ratio_S_u_diff(xi_val, rho_val, u_val, tau_val))
S_val_2_diff_u = Eq(Derivative(Seq_dbl_series.lhs,u), 
                    dbl_series_S(xi_val, rho_val, u_val, tau_val, M_val, N_val, _s=2))
S_val_1_diff_u
S_val_2_diff_u
abs(S_val_1_diff_u.rhs - S_val_2_diff_u.rhs) / abs(S_val_1_diff_u.rhs + S_val_2_diff_u.rhs) / 2

Eq(Derivative(Ser(xi, rho, u, tau), u), 1.38332108621588 - 1.14741674471763*I)

Eq(Derivative(Ser(xi, rho, u, tau), u), 1.38415855663739 - 1.1473988546033*I)

0.000116498738477364

### Kronecker-Eisenstein Series
Charollois and Sczech go on to introduce the following Kronecker-Eisenstein series (where lattice points that make the denominator zero should be ommitted). Where they sum over lattice points denoted $\lambda$; I explicitly write those points as $n \tau +m$, with $\bar{\lambda}=m+n\bar{\tau}$.

Note: I think Charollois and Sczech have a typo in the start of section 3 in their definition of the following function, I think they meant to write $\psi(\lambda \bar{w})$ instead of $\psi(z \bar{w})$ in the sum else the character could be factored out.

In [24]:
Kr_Es_dbl_series_K = Eq(K(z,w,s,tau,a), 
   Sum(Sum(
       (conjugate(z) + m + n * conjugate(tau)) ** a / 
       abs(z + m + n * tau) ** (2*s) 
       * exp(((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau))
    ,(n,-N,N)),(m,-M,M))
  )
re(s)>a/2+1
Kr_Es_dbl_series_K

re(s) > a/2 + 1

Eq(K(z, w, s, tau, a), Sum((m + n*conjugate(tau) + conjugate(z))**a*exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/Abs(m + n*tau + z)**(2*s), (n, -N, N), (m, -M, M)))

They then link this back to the double series developed earlier, namely $\mathrm{Ser}(\xi, \eta, u,\tau )$, by considering the following special case:

In [25]:
Kr_Es_dbl_series_K.subs([(a,1),(s,1)])
Eq(K(z, w, tau), Kr_Es_dbl_series_K.lhs.subs([(a,1),(s,1)]))
Eq(K(z, w, tau), Ser(xi, eta, z , tau))

Eq(K(z, w, 1, tau, 1), Sum((m + n*conjugate(tau) + conjugate(z))*exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/Abs(m + n*tau + z)**2, (n, -N, N), (m, -M, M)))

Eq(K(z, w, tau), K(z, w, 1, tau, 1))

Eq(K(z, w, tau), Ser(xi, eta, z, tau))

In [26]:
Kr_Es_dbl_series_K_a1_s1 = Eq(
    K(z, w, tau), 
    Sum(Sum(
        exp(((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau)) / (z + m + n * tau) 
    ,(n,-N,N)),(m,-M,M))
)
Kr_Es_dbl_series_K_a1_s1

Eq(K(z, w, tau), Sum(exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/(m + n*tau + z), (n, -N, N), (m, -M, M)))

Where $z, w \in \mathbb{C} $ (but they say $z$ cannot be a lattice point), and where $w$ is related to $\rho, \xi, \tau$ in the following way:

In [74]:
nm_character_arg_w_tau = (((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau))
w_tau_rho_xi = Eq(w,rho+xi*tau)
w_tau_rho_xi_conj = Eq(conjugate(w), rho +xi*conjugate(tau))
nm_character_arg_w_tau_rho_xi = Eq(
    nm_character_arg_w_tau,
    nm_character_arg_w_tau
    .subs([w_tau_rho_xi_conj.args, w_tau_rho_xi.args])
    .subs([(tau, re(tau) + I *im(tau))])
    .expand().factor()
)
rho_xi_sols_w_tau = solve([w_tau_rho_xi, w_tau_rho_xi_conj],[xi, rho])
rho_tau_w = Eq(rho, rho_xi_sols_w_tau[rho].simplify())
xi_tau_w = Eq(xi, rho_xi_sols_w_tau[xi].simplify())

Eq(im(xi),0)
Eq(im(rho),0)
w_tau_rho_xi
w_tau_rho_xi_conj
nm_character_arg_w_tau_rho_xi
rho_tau_w
xi_tau_w

Eq(im(xi), 0)

Eq(im(rho), 0)

Eq(w, rho + tau*xi)

Eq(conjugate(w), rho + xi*conjugate(tau))

Eq(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau), -2*I*pi*(m*xi - n*rho))

Eq(rho, (tau*conjugate(w) - w*conjugate(tau))/(tau - conjugate(tau)))

Eq(xi, (w - conjugate(w))/(tau - conjugate(tau)))

They then introduce a new function $\Theta$ and it is this function that they refer to as the *Kronecker theta function*. It is deifned as:

In [75]:
Eq(Theta(z, w, tau), K(z, w, tau)*exp(-z*conjugate(w)*pi/im(tau)))
Eq(Theta(z, w, tau), exp(-z*conjugate(w)*pi/im(tau))*Sum(Sum(
        exp(((m + n * tau)*conjugate(w) - (m + n * conjugate(tau))*w) * pi/im(tau)) / (z + m + n * tau) 
    ,(n,-N,N)),(m,-M,M)))

Eq(Theta(z, w, tau), K(z, w, tau)*exp(-pi*z*conjugate(w)/im(tau)))

Eq(Theta(z, w, tau), exp(-pi*z*conjugate(w)/im(tau))*Sum(exp(pi*(-w*(m + n*conjugate(tau)) + (m + n*tau)*conjugate(w))/im(tau))/(m + n*tau + z), (n, -N, N), (m, -M, M)))

In [76]:
Eq(Seq_dbl_ktheta.lhs.subs(u,z), 
   Seq_dbl_ktheta.rhs.subs([(u,z), (rho + tau*xi, w), xi_tau_w.subs(tau-conjugate(tau),2*I*im(tau)).args]))

Eq(Ser(xi, rho, z, tau), theta(w + z, tau)*theta(0, tau, 1)*exp(pi*z*(w - conjugate(w))/im(tau))/(theta(w, tau)*theta(z, tau)))

In [77]:
rho_xi_sols_w_tau = solve([w_tau_rho_xi, w_tau_rho_xi_conj],[xi, rho])
rho_tau_w = Eq(rho, rho_xi_sols_w_tau[rho].simplify())
xi_tau_w = Eq(xi, rho_xi_sols_w_tau[xi].simplify())

In [78]:
kf1 = ktheta_dbl_series.subs([
    xi_tau_w.subs(tau-conjugate(tau),2*I*im(tau)).args, rho_tau_w.subs(tau-conjugate(tau),2*I*im(tau)).args
])

In [79]:
Eq((kf1.lhs.subs(conjugate(tau), tau-2*I*im(tau)).simplify()*exp(-pi*u*(w-conjugate(w))/im(tau))).simplify(), 
   exp(-pi*u*(w-conjugate(w))/im(tau))*kf1.rhs)

Eq(theta(u + w, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(w, tau)), exp(-pi*u*(w - conjugate(w))/im(tau))*Sum(exp(2*I*pi*(I*m*(w - conjugate(w))/(2*im(tau)) - I*n*(tau*conjugate(w) - w*conjugate(tau))/(2*im(tau))))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [80]:
def kron_theta_ratio(_u, _w, _tau):
    result = (pi * 
        jtheta(1,(( _u + _w ) * pi).evalf(), exp(I*pi*_tau).evalf()) * 
        jtheta(1, 0, exp(I*pi*_tau).evalf(), 1) /
        (jtheta(1,( _u ) * pi.evalf(), exp(I*pi*_tau).evalf()) * 
         jtheta(1,( _w ) * pi.evalf(), exp(I*pi*_tau).evalf()) )
    ).evalf()
    return result

def kron_theta_series(_u, _w, _tau, _M, _N):
    
    _X = im(_w) / im(_tau)
    _Y = im(_tau * conjugate(_w)) / im(_tau)
    Total = (2*_M + 1)*(2*_N + 1)
    F = floor(Total/10)
    count = 0
    
    result = 0
    for _m in range(-_M, _M + 1):
        for _n in range(-_N, _N + 1):
            
            count += 1
            
            result += (exp(-2*I*pi*( _m * _X  - _n * _Y )) / (_m + _n * _tau +_u)).evalf()
            
            if not count % F:
                print(f"{floor(100*count/Total)+1}%")
                
    result *= exp(-2*I*pi*_u*im(_w)/im(_tau))
    
    return result.evalf()

In [81]:
w_val = rho_val + xi_val * tau_val

In [82]:
k_theta_ser_val = kron_theta_series(u_val, w_val, tau_val, 100, 100)
k_theta_ser_val

10%
20%
30%
40%
50%
60%
70%
80%
90%
100%


0.14476872365514 - 6.55569002600982*I

In [38]:
k_theta_ratio_val = kron_theta_ratio(u_val, w_val, tau_val)
k_theta_ratio_val

0.100887517211457 - 6.51923194210253*I

In [39]:
1-abs(k_theta_ratio_val/k_theta_ser_val)

0.00568462996931207

In [83]:
w_tau_conj_subs = [
    (w - conjugate(w), 2*I*im(w)), 
    (tau - conjugate(tau), 2*I*im(tau)),
    (tau*conjugate(w) - w*conjugate(tau), 2*I*im(tau*conjugate(w)))
]
exp_u_im_w_im_tau = exp(-2*I*pi*u*im(w)/im(tau))
ktheta_dbl_series_w_tau = ktheta_dbl_series.subs([rho_tau_w.args, xi_tau_w.args])
ktheta_dbl_series_w_tau = Eq(
    (ktheta_dbl_series_w_tau.lhs.simplify().subs(w_tau_conj_subs)*exp_u_im_w_im_tau).simplify(),
    ktheta_dbl_series_w_tau.rhs.subs(w_tau_conj_subs)*exp_u_im_w_im_tau
)
ktheta_dbl_series_w_tau

Eq(theta(u + w, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(w, tau)), exp(-2*I*pi*u*im(w)/im(tau))*Sum(exp(2*I*pi*(-m*im(w)/im(tau) + n*im(tau*conjugate(w))/im(tau)))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [84]:
Eq(im(x),0)
real_x_tau = ktheta_dbl_series_w_tau.subs(w,x*tau).subs([(im(x),0), (im(tau*x),x*im(tau))])
real_x_tau

Eq(im(x), 0)

Eq(theta(tau*x + u, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(tau*x, tau)), exp(-2*I*pi*u*x)*Sum(exp(-2*I*pi*m*x)/(m + n*tau + u), (n, -N, N), (m, -M, M)))

In [85]:
real_x_tau.subs(x, k/l)

Eq(theta(k*tau/l + u, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(k*tau/l, tau)), exp(-2*I*pi*k*u/l)*Sum(exp(-2*I*pi*k*m/l)/(m + n*tau + u), (n, -N, N), (m, -M, M)))

## Kronecker Theta Functions in Weierstrass Sigma Form

In this section the Kronecker theta function and the associated double sum pole expansion are given in the Weierstrass sigma form.

In [86]:
sigma_z_theta = Eq(sigma(z,g2,g3), 
   2*omega1/theta(0,tau,1)*exp(-(pi*z/2/omega1)**2 *theta(0,tau,3)/pi**3/6/theta(0,tau,1)*pi)*
   theta(z/2/omega1, tau)
)

zeta_omega1_theta = Eq(zw(omega1,g2,g3), - pi ** 2 * theta(0,tau,3)/pi**3 / ( 12 * omega1 * theta(0,tau,1)/pi))

kron_sigma_theta_relation = Eq(
    theta(u + w, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(w, tau)),
    2*omega1*sigma(2*omega1*(u + w), g2, g3) * 
    exp(-4*omega1*u*w*zw(omega1, g2, g3))/
    (sigma(2*omega1*u, g2, g3)*sigma(2*omega1*w, g2, g3))
)

sigma_z_theta
zeta_omega1_theta
kron_sigma_theta_relation

Eq(sigma(z, g2, g3), 2*omega1*theta(z/(2*omega1), tau)*exp(-z**2*theta(0, tau, 3)/(24*omega1**2*theta(0, tau, 1)))/theta(0, tau, 1))

Eq(zw(omega1, g2, g3), -theta(0, tau, 3)/(12*omega1*theta(0, tau, 1)))

Eq(theta(u + w, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(w, tau)), 2*omega1*sigma(2*omega1*(u + w), g2, g3)*exp(-4*omega1*u*w*zw(omega1, g2, g3))/(sigma(2*omega1*u, g2, g3)*sigma(2*omega1*w, g2, g3)))

In [87]:
w_x1_x2_tau = Eq(w, x1 + x2*tau)
u_x3_x4_omega = Eq(u, x3 + x4*tau)
ktheta_dbl_series_x_sigma = ktheta_dbl_series_w_tau.subs([
    kron_sigma_theta_relation.args,
    (im(w), x2*im(tau)), 
    (im(tau*conjugate(w)), x1*im(tau)), 
    w_x1_x2_tau.args,
    u_x3_x4_omega.args,
    (tau,omega2/omega1)
])
ktheta_dbl_series_x_sigma = Eq(
    ktheta_dbl_series_x_sigma.lhs/2/omega1,
    ktheta_dbl_series_x_sigma.rhs/2/omega1
).simplify()

kron_sigma_dbl_sum = Eq(
    Sum(exp(-2*I*pi*m*x2 + 2*I*pi*n*x1)/
    (2*omega1*(m + x3) + 2*omega2*(n + x4)), (n, -N, N), (m, -M, M)),
    sigma(2*omega1*(x1 + x3) + 2*omega2*(x2 + x4), g2, g3)*
    exp(-(2*omega1*x1 + 2*omega2*x2)*(2*omega1*x3 + 2*omega2*x4)*zw(omega1, g2, g3)/omega1)/
    (sigma(2*omega1*x1 + 2*omega2*x2, g2, g3)*sigma(2*omega1*x3 + 2*omega2*x4, g2, g3))*
    exp(I*pi*x2*(2*omega1*x3 + 2*omega2*x4)/omega1)
)


ktheta_dbl_series_w_tau
w_x1_x2_tau
u_x3_x4_omega
ktheta_dbl_series_x_sigma
kron_sigma_dbl_sum

Eq(theta(u + w, tau)*theta(0, tau, 1)/(theta(u, tau)*theta(w, tau)), exp(-2*I*pi*u*im(w)/im(tau))*Sum(exp(2*I*pi*(-m*im(w)/im(tau) + n*im(tau*conjugate(w))/im(tau)))/(m + n*tau + u), (n, -N, N), (m, -M, M)))

Eq(w, tau*x2 + x1)

Eq(u, tau*x4 + x3)

Eq(exp(-2*I*pi*x2*(x3 + omega2*x4/omega1))*Sum(exp(-2*I*pi*m*x2 + 2*I*pi*n*x1)/(m*omega1 + n*omega2 + omega1*x3 + omega2*x4), (n, -N, N), (m, -M, M))/2, sigma(2*omega1*(x1 + x3) + 2*omega2*x2 + 2*omega2*x4, g2, g3)*exp(-4*(omega1*x1 + omega2*x2)*(omega1*x3 + omega2*x4)*zw(omega1, g2, g3)/omega1)/(sigma(2*omega1*x1 + 2*omega2*x2, g2, g3)*sigma(2*omega1*x3 + 2*omega2*x4, g2, g3)))

Eq(Sum(exp(-2*I*pi*m*x2 + 2*I*pi*n*x1)/(2*omega1*(m + x3) + 2*omega2*(n + x4)), (n, -N, N), (m, -M, M)), sigma(2*omega1*(x1 + x3) + 2*omega2*(x2 + x4), g2, g3)*exp((-2*omega1*x1 - 2*omega2*x2)*(2*omega1*x3 + 2*omega2*x4)*zw(omega1, g2, g3)/omega1)*exp(I*pi*x2*(2*omega1*x3 + 2*omega2*x4)/omega1)/(sigma(2*omega1*x1 + 2*omega2*x2, g2, g3)*sigma(2*omega1*x3 + 2*omega2*x4, g2, g3)))

### Accelerated convergence using the difference between the function evaluated in two places
The series can be made to converge faster by subtracting the function evaluated at one point from the function at evaluated at another point and combining the terms in the sum to be quadratic in the denominator rather than linear. This helps test the equations numerically because convergence is generally quite slow.

In [150]:
two_terms = 1/(2*omega1*(m + x3) + 2*omega2*(n + x4)) - 1/(2*omega1*(m + x5) + 2*omega2*(n + x6))
_n,_d = fraction(simplify(two_terms))
Eq(
    two_terms,
    _n.simplify().collect([omega1, omega2])/_d
)

Eq(-1/(2*omega1*(m + x5) + 2*omega2*(n + x6)) + 1/(2*omega1*(m + x3) + 2*omega2*(n + x4)), (omega1*(-x3 + x5) + omega2*(-x4 + x6))/(2*(omega1*(m + x3) + omega2*(n + x4))*(omega1*(m + x5) + omega2*(n + x6))))

In [459]:
kron_diff = Eq(
    kron_sigma_dbl_sum.rhs / (2*omega1*(x5 - x3) + 2*omega2*(x6 - x4)) - 
    kron_sigma_dbl_sum.rhs.subs([(x3,x5),(x4,x6)]) / (2*omega1*(x5 - x3) + 2*omega2*(x6 - x4))
    ,
    Sum(
        exp(-2*I*pi*m*x2 + 2*I*pi*n*x1)/
        (2*omega1*(m + x3) + 2*omega2*(n + x4))/
        (2*omega1*(m + x5) + 2*omega2*(n + x6))
    , (n, -N, N), (m, -M, M))
)
kron_diff

Eq(-sigma(2*omega1*(x1 + x5) + 2*omega2*(x2 + x6), g2, g3)*exp((-2*omega1*x1 - 2*omega2*x2)*(2*omega1*x5 + 2*omega2*x6)*zw(omega1, g2, g3)/omega1)*exp(I*pi*x2*(2*omega1*x5 + 2*omega2*x6)/omega1)/((2*omega1*(-x3 + x5) + 2*omega2*(-x4 + x6))*sigma(2*omega1*x1 + 2*omega2*x2, g2, g3)*sigma(2*omega1*x5 + 2*omega2*x6, g2, g3)) + sigma(2*omega1*(x1 + x3) + 2*omega2*(x2 + x4), g2, g3)*exp((-2*omega1*x1 - 2*omega2*x2)*(2*omega1*x3 + 2*omega2*x4)*zw(omega1, g2, g3)/omega1)*exp(I*pi*x2*(2*omega1*x3 + 2*omega2*x4)/omega1)/((2*omega1*(-x3 + x5) + 2*omega2*(-x4 + x6))*sigma(2*omega1*x1 + 2*omega2*x2, g2, g3)*sigma(2*omega1*x3 + 2*omega2*x4, g2, g3)), Sum(exp(-2*I*pi*m*x2 + 2*I*pi*n*x1)/((2*omega1*(m + x3) + 2*omega2*(n + x4))*(2*omega1*(m + x5) + 2*omega2*(n + x6))), (n, -N, N), (m, -M, M)))

### Numerical test of the Weierstrass sigma form of Kronecker theta functions

In [479]:
def kron_sigma_sol(_x1, _x2, _x3, _x4, _x5, _x6, _omega1, _omega2):

    omegas = (_omega1, _omega2) 
    result = (
        exp(-(2*_omega1*_x1 + 2*_omega2*_x2)*(2*_omega1*_x5 + 2*_omega2*_x6)*we.wzeta(_omega1, omegas)/_omega1)*
        exp(I*pi*_x2*(2*_omega1*_x5 + 2*_omega2*_x6)/_omega1)*
        we.wsigma(2*_omega1*(_x1 + _x5) + 2*_omega2*(_x2 + _x6), omegas) / 
        we.wsigma(2*_omega1*_x1 + 2*_omega2*_x2, omegas) /
        we.wsigma(2*_omega1*_x5 + 2*_omega2*_x6, omegas) /
        (2*_omega1*(_x3 - _x5) + 2*_omega2*(_x4 - _x6)) -
        
        exp(-(2*_omega1*_x1 + 2*_omega2*_x2)*(2*_omega1*_x3 + 2*_omega2*_x4)*we.wzeta(_omega1, omegas)/_omega1)*
        exp(I*pi*_x2*(2*_omega1*_x3 + 2*_omega2*_x4)/_omega1)*
        we.wsigma(2*_omega1*(_x1 + _x3) + 2*_omega2*(_x2 + _x4), omegas) / 
        we.wsigma(2*_omega1*_x1 + 2*_omega2*_x2, omegas) /
        we.wsigma(2*_omega1*_x3 + 2*_omega2*_x4, omegas) /
        (2*_omega1*(_x3 - _x5) + 2*_omega2*(_x4 - _x6))
    ).evalf()
    return result

def kron_theta_series_sigma(_x1, _x2, _x3, _x4, _x5, _x6, _omega1, _omega2, _M, _N):
    
    # Track progress
    Total = (2*_M + 1)*(2*_N + 1)
    F = floor(Total/10)
    count = 0
    
    result = 0
    for _m in range(-_M, _M + 1):
        for _n in range(-_N, _N + 1):
            
            count += 1
            
            result += (
                exp(2*I*pi*(- _m * _x2  + _n * _x1 )) / 
                (2*_omega1*(_m + _x3) + 2*_omega2*(_n + _x4)) /
                (2*_omega1*(_m + _x5) + 2*_omega2*(_n + _x6))
            ).evalf()
            
            if not count % F:
                print(f"{floor(100*count/Total)+1}%")
    
    return result.evalf()

def kron_sigma_sol_1(_x1, _x2, _x3, _x4, _u, _omega1, _omega2):

    omegas = (_omega1, _omega2) 
    result = (
        exp(-(2*_omega1*_x1 + 2*_omega2*_x2)*_u*we.wzeta(_omega1, omegas)/_omega1)*
        exp(I*pi*_x2*_u/_omega1)*
        we.wsigma(2*_omega1*_x1 + 2*_omega2*_x2 + _u, omegas) / 
        we.wsigma(2*_omega1*_x1 + 2*_omega2*_x2, omegas) /
        we.wsigma(_u, omegas)
    ).evalf()
    return result

def kron_theta_series_sigma_1(_x1, _x2, _x3, _x4, _u, _omega1, _omega2, _M, _N):
    
    # Track progress
    Total = (2*_M + 1)*(2*_N + 1)
    F = floor(Total/10)
    count = 0
    
    result = 0
    for _m in range(-_M, _M + 1):
        for _n in range(-_N, _N + 1):
            
            count += 1      
            result += (exp(2*I*pi*(- _m * _x2  + _n * _x1 )) / (2*_omega1*_m + 2*_omega2*_n + _u)).evalf()
            
            if not count % F:
                print(f"{floor(100*count/Total)+1}%")
    
    return result.evalf()

In [492]:
in_vals = {
    '_x1': 0.245, 
    '_x2': 0.2345,
    '_x3': 0.3256,
    '_x4': 0.15,
    '_x5': 0.988,
    '_x6': 0.489,
    '_omega1': 1.89+3.2j,
    '_omega2': -2.35+4.3j,
    '_M': 150,
    '_N': 150
}

if im(in_vals['_omega2']/in_vals['_omega1']) <= 0:
    print('Woops! make omega2 negative')
    
in_vals_no_MN = {k:in_vals[k] for k in in_vals if k not in ['_M', '_N']}

r_1 = kron_theta_series_sigma(**in_vals)
r_2 = kron_sigma_sol(**in_vals_no_MN)

r_1
r_2
2*abs(r_1 - r_2)/abs(r_1 + r_2)

10%
20%
30%
40%
50%
60%
70%
80%
90%
100%


-0.0450182840957089 + 0.0352487459692174*I

-0.045018192792082 + 0.0352489198981145*I

3.43564588973059e-6

In [ ]:
# in_vals = {
#     '_x1': 0.145, 
#     '_x2': 0.2345,
#     '_x3': 0.1256,
#     '_x4': 0.15,
#     '_x5': 0.088,
#     '_x6': 0.189,
#     '_omega1': 0.89+1.2j,
#     '_omega2': -2.35+0.2j,
#     '_M': 200,
#     '_N': 200
# }
# 0.000235756741884392  M = N = 10
# 2.18717328312761⋅10−5 M = N = 40
# 3.48144971782924⋅10−6 M = N = 70
# 4.98448487389926⋅10−6 M = N = 100
# 1.72375801839293⋅10−6 M = N = 130
# 1.24514498542695⋅10−6 M = N = 200


# in_vals = {
#     '_x1': 0.245, 
#     '_x2': 0.2345,
#     '_x3': 0.3256,
#     '_x4': 0.15,
#     '_x5': 0.988,
#     '_x6': 0.489,
#     '_omega1': 1.89+3.2j,
#     '_omega2': -2.35+4.3j,
#     '_M': 10,
#     '_N': 10
# }
# M = N = 10, 0.00263409240388377
# M = N = 40, 0.000169611151416672
# M = N = 70, 1.61380137365884⋅10−5
# M = N = 150, 3.43564588973059⋅10−6

In [482]:
in_vals_2 = {
    '_x1': 0.145, 
    '_x2': 0.2345,
    '_x3': 0.1256,
    '_x4': 0.15,
    '_u': 1.88,
    '_omega1': 0.89+1.2j,
    '_omega2': -2.35+0.2j,
    '_M': 100,
    '_N': 100
}

if im(in_vals_2['_omega2']/in_vals_2['_omega1']) <= 0:
    print('Woops! make omega2 negative')
    
in_vals_no_MN_2 = {k:in_vals_2[k] for k in in_vals_2 if k not in ['_M', '_N']}
r_1 = kron_theta_series_sigma_1(**in_vals_2)

r_2 = kron_sigma_sol_1(**in_vals_no_MN_2)

r_1
r_2
2*abs(r_1 - r_2)/abs(r_1 + r_2)

10%
20%
30%
40%
50%
60%
70%
80%
90%
100%


-0.406342794784371 - 0.681989373868458*I

-0.406938469620351 - 0.683674247780049*I

0.00224861800000889

In [ ]:
# in_vals_2 = {
#     '_x1': 0.145, 
#     '_x2': 0.2345,
#     '_x3': 0.1256,
#     '_x4': 0.15,
#     '_u': 1.88,
#     '_omega1': 0.89+1.2j,
#     '_omega2': -2.35+0.2j,
#     '_M': 100,
#     '_N': 100
# }

# 0.00841984123373364 M = N = 40 
# 0.00609657110821202 M = N = 70 
# 0.00224861800000889 M = N = 100 

## The Kronecker Second Limit Formula

This is taken from Equation (7) in Charollois and Sczech but there is no derivation in that reference nor is there a citation to its derivation. I have found a derivation in Siegel [Lectures on advanced analytic number theory](https://www-users.cse.umn.edu/~garrett/m/mfms/notes_2013-14/Siegel_AdvAnNoTh.pdf)

In [57]:
im(tau)>0
im(kappa)<0
Eq(im(xi),0)
Eq(im(rho),0)

im(tau) > 0

im(kappa) < 0

Eq(im(xi), 0)

Eq(im(rho), 0)

In [204]:
Eq(Sum((tau - kappa)/(2*pi*I)*exp(2*I*pi*(n*xi +m*rho))/(n + m*tau)/(n+m*kappa),(n,-oo,oo),(m,-oo,oo)),
  -ln(theta(rho - xi*tau,tau)*theta(rho - xi*kappa,-kappa)/eta(tau)/eta(-kappa)) -pi*I*xi**2*(tau - kappa)
  )

Eq(Sum(-I*(-kappa + tau)*exp(2*I*pi*(m*rho + n*xi))/(2*pi*(kappa*m + n)*(m*tau + n)), (n, -oo, oo), (m, -oo, oo)), -I*pi*xi**2*(-kappa + tau) - log(theta(rho - tau*xi, tau)*theta(-kappa*xi + rho, -kappa)/(eta(-kappa)*eta(tau))))

$\theta(z,\tau,k)=\pi^k \mathrm{jtheta}(1,\pi z,\exp(i \pi \tau),k)$

In [201]:
def kron_second_limit_formula_sum(_xi, _rho, _tau, _kappa, _M, _N):
    
    # Track progress
    Total = (2*_M + 1)*(2*_N + 1)
    F = floor(Total/10)
    count = 0
    
    result = 0
    for _m in range(-_M, _M + 1):
        for _n in range(-_N, _N + 1):
            
            count += 1
            
            if not ((_m == 0) and (_n == 0)):
            
                result += (
                    exp(2*I*pi*(_m * _xi  + _n * _rho )) / 
                    (_m + _n * _tau) /
                    (_m + _n * _kappa)
                ).evalf()
            
            if not count % F:
                print(f"{floor(100*count/Total)+1}%")
    
    return (result / (2*I*pi) / (_tau - _kappa)).evalf()

def dedekind_eta(_tau, _DN):
#     q = exp(2*I*pi*_tau)
#     return (q**(1/24)*product(1 - q**n,(n,1,_DN))).evalf()
    return (exp(2*I*pi*_tau/24) * product(1 - exp(2*I*pi*n*_tau), (n,1,_DN))).evalf()

def kron_second_limit_formula_log_theta(_xi, _rho, _tau, _kappa, _DN):
    result = (
        -I*pi*_xi**2*(_tau - _kappa)
        -ln(
            jtheta(1, ((_rho - _tau * _xi) * pi).evalf(), exp(I*pi*_tau).evalf() ) *
            jtheta(1, ((_rho - _kappa * _xi) * pi).evalf(), exp(-I*pi*_kappa).evalf() ) /
            dedekind_eta(_tau, _DN) /
            dedekind_eta(-_kappa, _DN)
        )
    )
    return result.evalf()

In [97]:
(1/(m + n*tau) -1/(m+n*kappa)).simplify()

n*(kappa - tau)/((m + n*tau)*(kappa*n + m))

In [202]:
in_vals_kl = {
    '_xi': 0.0,
    '_rho': 0.165, 
    '_tau': 1.1256 + 2.345j,
    '_kappa': 0.15 - 2.4456j,
    '_M': 100,
    '_N': 100,
    '_DN': 200
}

kl_r_1 = kron_second_limit_formula_sum(**{k:in_vals_kl[k] for k in in_vals_kl if k not in ['_DN']})
kl_r_2 = kron_second_limit_formula_log_theta(**{k:in_vals_kl[k] for k in in_vals_kl if k not in ['_M','_N']})

kl_r_1
kl_r_2
2*abs(kl_r_1 - kl_r_2)/abs(kl_r_1 + kl_r_2)

10%
20%
30%
40%
50%
60%
70%
80%
90%
100%


-0.105567708560065 - 0.0216186989298628*I

2.52660103151331 + 1.0599734750031*I

2.16051020431978

In [200]:
kl_r_2 = kron_second_limit_formula_log_theta(**{k:in_vals_kl[k] for k in in_vals_kl if k not in ['_M','_N']})

kl_r_1
kl_r_2
2*abs(kl_r_1 - kl_r_2)/abs(kl_r_1 + kl_r_2)

-0.021102598481314 - 0.00815318627109102*I

2.52660103151331 + 1.0599734750031*I

2.03328813273865

In [176]:
dedekind_eta(1.3+2.34j, 400)

0.510849859365029 + 0.18090119779838*I

In [ ]:
0.510849859365029+0.18090119779838𝑖

0.5402637477889+0.0425194661684689𝑖

In [205]:
(gamma(1/4)/(2*pi**(3/4))).evalf()
(gamma(1/4)/(2**(7/8)*pi**(3/4))).evalf()

0.768225422326057

0.837755763476598

In [206]:
dedekind_eta(1j, 10000)
dedekind_eta(0.5j, 10000)

0.768225434313799

0.837755770012961

In [ ]:
0.768225422302643

0.768225434313799

In [194]:
(-pi*I*((rho - xi*tau) - (rho - xi*kappa))**2/(tau - kappa)).simplify()

I*pi*xi**2*(kappa - tau)